In [1]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
from collections import Counter

from math import radians, sin, cos, sqrt, atan2

In [2]:
import importlib
import tonelab.tone2vec
importlib.reload(tonelab.tone2vec)
from tonelab.tone2vec import loading, parse_phonemes, tone_feats, plot

dataset_path, dataset_info = 'tests/examples/dataset.csv', 'tests/examples/info.csv' 
dataset, labels = loading(dataset_path), loading(dataset_info, column_name='areas')
initial_list, final_list, all_list,  tone_list = parse_phonemes(dataset)
feats = tone_feats(tone_list)
plot(feats, labels, 'PCA')

/home/yangyi/anaconda3/envs/ToneLab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


There are 82 unique phonemes
Filtered phonemes: ['t', 'o', 'h', 'ɑ', 'l', 's', 'k', 'ŋ', 'x', 'ɕ', 'i', 'e', 'p', 'm', 'ʂ', 'u', 'Ǿ', 'n', 'ȵ', 'ʯ', 'ʐ', 'f', 'ɚ', 'æ', 'ʅ', 'ɿ', 'ə', '̃', 'a', 'r', 'd', 'ɡ', 'b', 'z', 'y', 'ʑ', 'ɪ', 'ʔ', 'ɛ', 'ɔ', 'ɤ', 'ᴀ', 'v', 'ɦ', 'ᴇ', 'ɐ', 'ɵ', 'ɥ', 'ʃ', 'ʮ', 'ʊ', 'ɾ', 'ɒ', 'ɘ', 'ã', 'ẽ', 'ʒ', 'ɣ', 'ɞ', 'j', 'ᴂ', 'õ', 'ʉ', 'ɨ', 'ɻ', 'ʌ', 'ɯ', 'ø', 'ı', '∅', 'œ', 'ɬ', 'θ', 'Ø', 'ʦ', 'ʨ', 'g', 'ʏ', 'ɜ', 'ɸ', 'c', 'ʷ']


In [7]:
import os
import umap
import math
import pickle
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from typing import List
from torch import optim
from random import sample
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.models as models
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from scipy.optimize import linear_sum_assignment
from sklearn.manifold import TSNE, MDS, Isomap
from sklearn.neighbors import KNeighborsRegressor
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from matplotlib.colors import LinearSegmentedColormap
from scipy.cluster.hierarchy import linkage, fcluster
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [ ]:
import os
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingLR

def train_and_evaluate(model, train_loader, valid_loader, optimizer, device, num_epochs, save_dir, unique_transcription, print_interval):
    """
    Train the model on the training set and select the best performing model on the validation set.

    Args:
    - model: The neural network model to train.
    - train_loader: DataLoader for the training dataset.
    - valid_loader: DataLoader for the validation dataset.
    - optimizer: Optimizer for updating the model parameters.
    - device: Device to run the training on (e.g., 'cpu' or 'cuda').
    - num_epochs: Number of training epochs.
    - save_dir: Directory to save the best model and training logs.
    - unique_transcription: Tensor containing unique transcriptions for evaluation.
    - print_interval: int, interval for printing the logs.
    """

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    log_file_path = os.path.join(save_dir, 'training_log.txt')
    unique_transcription = unique_transcription.to(device)
    criterion = nn.L1Loss()

    with open(log_file_path, 'w') as log_file:
        scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
        best_valid_accuracy = 0

        for epoch in range(num_epochs):
            model.train()
            train_loss_accum = 0

            for batch_X, batch_y in train_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                optimizer.zero_grad()
                values = model(batch_X)
                loss = criterion(values, batch_y)
                loss.backward()
                optimizer.step()
                train_loss_accum += loss.item()

            scheduler.step()
            log_file.write(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss_accum / len(train_loader):.4f}\n')

            model.eval()
            train_correct, valid_correct = 0, 0

            with torch.no_grad():
                for batch_X, batch_y in train_loader:
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                    values = model(batch_X)
                    train_correct += eval_metric(values, batch_y, unique_transcription, metric_type='acc')

                for batch_X, batch_y in valid_loader:
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                    values = model(batch_X)
                    valid_correct += eval_metric(values, batch_y, unique_transcription, metric_type='acc')

            valid_accuracy = valid_correct / len(valid_loader.dataset)
            train_accuracy = train_correct / len(train_loader.dataset)

            log_file.write(f'Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.4f}, Valid Accuracy: {valid_accuracy:.4f}\n')

            if (epoch + 1) % print_interval == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.4f}, Valid Accuracy: {valid_accuracy:.4f}\n')

            if valid_accuracy > best_valid_accuracy:
                best_valid_accuracy = valid_accuracy
                best_model_path = os.path.join(save_dir, 'best_model.pth')
                torch.save(model.state_dict(), best_model_path)
                log_file.write(f"Saved Best Model at Epoch {epoch+1} with Valid Accuracy: {best_valid_accuracy:.4f}\n")

            log_file.flush()

import os
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingLR

def testing(model, test_feat, test_label, unique_transcription, checkpoint_dir, device):
    """
    Evaluate the model on the test set.

    Args:
    - model: The neural network model to train.
    """

    best_model_path = os.path.join(checkpoint_dir, 'best_model.pth')
    model_state_dict = torch.load(best_model_path, map_location=device)
    model.load_state_dict(model_state_dict)

    unique_transcription = unique_transcription.to(device)

    model.eval()
    with torch.no_grad():
        feats, labels = test_feat.to(device), test_label.to(device)
        values = model(feats)

        accuracy = eval_metric(values, labels, unique_transcription, metric_type='acc')
        loss   = eval_metric(values, labels, unique_transcription, metric_type='mae')

        return accuracy/test_feat.shape[0], loss/test_feat.shape[0]


def eval_metric(values, batch_y, unique_transcription, metric_type='acc'):
    """
    Calculate the accuracy or MAE of the predictions.

    Args:
    - values: torch.tensor of shape (batch_size, 3), the predicted values.
    - batch_y: torch.tensor of shape (batch_size, 3), the true labels.
    - unique_transcription: torch.tensor of shape (n, 3), the tensor containing unique transcriptions.
    - metric_type: str, the type of metric to calculate ('acc' for accuracy, 'mae' for mean absolute error).

    Returns:
    - int or float, the calculated metric (number of correct predictions for accuracy, total MAE for mean absolute error).
    """
    correct = 0
    total_loss = 0.0

    for index in range(batch_y.size(0)):
        signal = batch_y[index]
        value = values[index]
        pred_seq = match_transcription(value, unique_transcription)

        with torch.no_grad():
            if torch.sum(torch.abs(pred_seq - signal)) < 1e-2:
                correct += 1
            total_loss += torch.sum(torch.abs(pred_seq - signal))

    if metric_type == 'acc':
        return correct
    elif metric_type == 'mae':
        return total_loss